In [1]:
from tensorflow import keras
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras import backend as K
import os
import numpy as np
import json
from collections import Counter
from tensorflow.keras.optimizers import SGD

In [2]:
pet_images_fn = [fn for fn in os.listdir('pet_images') if fn.endswith('.jpg')]
labels = []
idx_to_labels = []
label_to_idx = {}
for fn in pet_images_fn:
    label, _ = fn.rsplit('_', 1)
    if not label in label_to_idx:
        label_to_idx[label] = len(idx_to_labels)
        idx_to_labels.append(label)
    labels.append(label_to_idx[label])
len(idx_to_labels)

37

In [3]:
def fetch_pet(pet):
    img = image.load_img('pet_images/' + pet, target_size=(299, 299))
    return image.img_to_array(img)

img_vector = np.asarray([fetch_pet(pet) for pet in pet_images_fn])

In [8]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
for layer in base_model.layers:
    layer.trainable = False

pool_2d = GlobalAveragePooling2D(name='pool_2d')(base_model.output)
# dense = Dense(1024, name='dense', activation='relu')(pool_2d)
dense = Dense(1024, activation='relu')(pool_2d)
predictions = Dense(len(idx_to_labels), activation='softmax')(dense)
model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_376 (Conv2D)             (None, 149, 149, 32) 864         <tensorflow.python.keras.engine.i
__________________________________________________________________________________________________
batch_normalization_v1_376 (Bat (None, 149, 149, 32) 96          <tensorflow.python.keras.layers.c
__________________________________________________________________________________________________
activation_376 (Activation)     (None, 149, 149, 32) 0           <tensorflow.python.keras.layers.n
__________________________________________________________________________________________________
conv2d_377

In [9]:
# 0.9448
y = np.zeros((len(labels), len(idx_to_labels)))
for idx, label in enumerate(labels):
    y[idx][label] = 1

model.fit(
    img_vector, y,
    batch_size=128,
    epochs=15,
    verbose=2
)

Epoch 1/15
 - 56s - loss: 1.4764 - acc: 0.6606
Epoch 2/15
 - 43s - loss: 0.5172 - acc: 0.8419
Epoch 3/15
 - 43s - loss: 0.3824 - acc: 0.8777
Epoch 4/15
 - 43s - loss: 0.2939 - acc: 0.9003
Epoch 5/15
 - 43s - loss: 0.2490 - acc: 0.9153
Epoch 6/15
 - 43s - loss: 0.1952 - acc: 0.9334
Epoch 7/15
 - 43s - loss: 0.1704 - acc: 0.9414
Epoch 8/15
 - 43s - loss: 0.1530 - acc: 0.9467
Epoch 9/15
 - 43s - loss: 0.1328 - acc: 0.9568
Epoch 10/15
 - 43s - loss: 0.1104 - acc: 0.9609
Epoch 11/15
 - 43s - loss: 0.0962 - acc: 0.9671
Epoch 12/15
 - 43s - loss: 0.0823 - acc: 0.9687
Epoch 13/15
 - 43s - loss: 0.0738 - acc: 0.9758
Epoch 14/15
 - 43s - loss: 0.0633 - acc: 0.9778
Epoch 15/15
 - 43s - loss: 0.0622 - acc: 0.9792


In [10]:
unfreeze = False
for layer in base_model.layers:
    if unfreeze:
        layer.trainable = True
    if layer.name == 'mixed9':
        unfreeze = True
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), 
              loss='categorical_crossentropy', metrics=['accuracy'])

In [11]:
model.fit(
    img_vector, y,
    batch_size=128,
    epochs=10,
    verbose=2
)

Instructions for updating:
Use tf.cast instead.


ValueError: Incompatible shapes between op input and calculated input gradient.  Forward operation: cond_922.  Input index: 2. Original input shape: (None, 8, 8, 320).  Calculated input gradient shape: (320,)

In [ ]:
json.dump(idx_to_labels, open('zoo/09.3 pet_labels.json', 'w'))
model.save('zoo/09.3 retrained pet recognizer.h5')
